In [1]:
!wget --no-check-certificate https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Luxury_Beauty.json.gz

--2022-12-17 16:52:05--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Luxury_Beauty.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 73193215 (70M) [application/x-gzip]
Saving to: ‘Luxury_Beauty.json.gz’

Luxury_Beauty.json. 100%[===================>]  69.80M  18.0MB/s    in 5.1s    

2022-12-17 16:52:11 (13.8 MB/s) - ‘Luxury_Beauty.json.gz’ saved [73193215/73193215]



In [2]:
!wget --no-check-certificate https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Luxury_Beauty.json.gz

--2022-12-17 16:52:11--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Luxury_Beauty.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 7608964 (7.3M) [application/x-gzip]
Saving to: ‘meta_Luxury_Beauty.json.gz’

meta_Luxury_Beauty. 100%[===================>]   7.26M  4.74MB/s    in 1.5s    

2022-12-17 16:52:13 (4.74 MB/s) - ‘meta_Luxury_Beauty.json.gz’ saved [7608964/7608964]



In [3]:
from pymystem3 import Mystem
mystem = Mystem()
from string import punctuation

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [4]:
!pip install pymystem3==0.1.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [5]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2022-12-17 16:52:39--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cachev2-spb03.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122 [following]
--2022-12-17 16:52:42--  http://cachev2-spb03.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122
Resolving cachev2-spb03.cdn.yandex.net (cachev2-spb03.cdn.yandex.net)... 37.140.137.3, 2a02:6b8:0:2221::303
Connecting to cachev2-spb03.cdn.yandex.net (cachev2-spb03.cdn.yandex.net)|37.140.137.3|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[=====

In [6]:
import pandas as pd
import gzip
import json
import spacy
nlp = spacy.load('en_core_web_sm')
from tqdm.auto import tqdm
import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Читаем данные

In [7]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Luxury_Beauty.json.gz')
df_meta = getDF('meta_Luxury_Beauty.json.gz')

In [8]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,2.0,3,True,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,1276560000,NaN,NaN
1,5.0,14,True,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,1262822400,NaN,NaN
2,5.0,NaN,True,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN
3,5.0,NaN,True,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN
4,5.0,NaN,True,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,1523923200,{'Size:': ' 3.5 oz.'},NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574628 entries, 0 to 574627
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         574628 non-null  float64
 1   vote            103689 non-null  object 
 2   verified        574628 non-null  bool   
 3   reviewTime      574628 non-null  object 
 4   reviewerID      574628 non-null  object 
 5   asin            574628 non-null  object 
 6   reviewerName    574597 non-null  object 
 7   reviewText      574228 non-null  object 
 8   summary         574445 non-null  object 
 9   unixReviewTime  574628 non-null  int64  
 10  style           251013 non-null  object 
 11  image           7418 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 53.2+ MB


In [10]:
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[After a long day of handling thorny situation...,,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"[B00GHX7H0A, B00FRERO7G, B00R68QXCS, B000Z65AZ...",,,[],"4,324 in Beauty & Personal Care (","[B00FRERO7G, B00GHX7H0A, B07GFHJRMX, B00TJ3NBN...",{' Product Dimensions: ': '2.2 x 2.2 ...,Luxury Beauty,,,$30.00,B00004U9V2,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[If you haven't experienced the pleasures of b...,,AHAVA Bath Salts,[],,,[],"1,633,549 in Beauty & Personal Care (",[],{' Product Dimensions: ': '3 x 3.5 x ...,Luxury Beauty,,,,B0000531EN,[],[]
2,[],,"[Rich, black mineral mud, harvested from the b...",,"AHAVA Dead Sea Mineral Mud, 8.5 oz, Pack of 4",[],,,[],"1,806,710 in Beauty &amp; Personal Care (",[],{' Product Dimensions: ': '5.1 x 3 x ...,Luxury Beauty,,,,B0000532JH,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[This liquid soap with convenient pump dispens...,,"Crabtree &amp; Evelyn Hand Soap, Gardeners, 10...",[],,,[],[],"[B00004U9V2, B00GHX7H0A, B00FRERO7G, B00R68QXC...",{' Product Dimensions: ': '2.6 x 2.6 ...,Luxury Beauty,,,$15.99,B00005A77F,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Remember why you love your favorite blanket? ...,,Soy Milk Hand Crme,"[B000NZT6KM, B001BY229Q, B008J724QY, B0009YGKJ...",,,[],"42,464 in Beauty &amp; Personal Care (",[],{' Product Dimensions: ': '7.2 x 2.2 ...,Luxury Beauty,,,$18.00,B00005NDTD,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [11]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12299 entries, 0 to 12298
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         12299 non-null  object
 1   tech1            12299 non-null  object
 2   description      12299 non-null  object
 3   fit              12299 non-null  object
 4   title            12299 non-null  object
 5   also_buy         12299 non-null  object
 6   tech2            12299 non-null  object
 7   brand            12299 non-null  object
 8   feature          12299 non-null  object
 9   rank             12299 non-null  object
 10  also_view        12299 non-null  object
 11  details          12299 non-null  object
 12  main_cat         12299 non-null  object
 13  similar_item     12299 non-null  object
 14  date             12299 non-null  object
 15  price            12299 non-null  object
 16  asin             12299 non-null  object
 17  imageURL         12299 non-null

## Какими способами можно найти упоминания товаров в отзывах?

1. Использовать tf-idf, где в качестве документа будет коллекция отзывов нв товар. Tf-idf покажет нам уникальные слова для товара. Оставим из них только существительные, но это могут быть существительные не только имеющие название товара, но и показывающие какие-то свойства, и нам надо их разделить, чтобы оставить только упоиминания товаров. Думаю, можно воспользоваться кластеризацией, разделить по семантическим признакам существительные и получить отдельно кластер с названиями товаров. Этот способ хорош тем, что нам не нужно делать много ручной работы, при этом в теории кластеры помогут с отделением NE от не NE. Проблема в том, что мы не можем быть уверены, что NE у нас разметятся правильно

2. Можно посмотреть на самые частотные слова из отзывов для каждого из товаров, иногда наверное даже на биграммы. Думаю, можно взять не все отзывы, а какую-то часть, на которой можно это потренировать. Мое предположение в том, что, например для одного товара (допустим, крема для рук) часто будут в отзывах встречаться слова "cream" или "hand cream", а это то, что нам нужно. Далее можно найти пересечение самых частотных слов в тайтлах и самых частотных словах в отзывах. Это по идее и должны быть наши названия товаров. Конечно, тут тоже может попадаться мусор, но думаю так его будет меньше. 

3. Можно использовать word2vec, взять близкие по семантике слова к товарам красоты, или найти аналогии с самыми частотными словами в отзывах.

## Реализация

Я постараюсь реализовать второй способ

In [12]:
df_reviews = df[['asin', 'reviewText']]

In [13]:
df_reviews

,asin,reviewText
0,B00004U9V2,"I bought two of these 8.5 fl oz hand cream, an..."
1,B00004U9V2,"Believe me, over the years I have tried many, ..."
2,B00004U9V2,Great hand lotion
3,B00004U9V2,This is the best for the severely dry skin on ...
4,B00004U9V2,The best non- oily hand cream ever. It heals o...
...,...,...
574623,B01HIQEOLO,Great color and I prefer shellac over gel
574624,B01HIQEOLO,Best shellac I have ever used. It doesn't tak...
574625,B01HIQEOLO,Great polish and beautiful color!!
574626,B01HJ2UY0W,"The perfume is good, but the spray head broke ..."


In [14]:
df_reviews.dropna()

,asin,reviewText
0,B00004U9V2,"I bought two of these 8.5 fl oz hand cream, an..."
1,B00004U9V2,"Believe me, over the years I have tried many, ..."
2,B00004U9V2,Great hand lotion
3,B00004U9V2,This is the best for the severely dry skin on ...
4,B00004U9V2,The best non- oily hand cream ever. It heals o...
...,...,...
574623,B01HIQEOLO,Great color and I prefer shellac over gel
574624,B01HIQEOLO,Best shellac I have ever used. It doesn't tak...
574625,B01HIQEOLO,Great polish and beautiful color!!
574626,B01HJ2UY0W,"The perfume is good, but the spray head broke ..."


In [15]:
df_meta[df_meta['asin']=='B00005A77F']

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
3,[],,[This liquid soap with convenient pump dispens...,,"Crabtree &amp; Evelyn Hand Soap, Gardeners, 10...",[],,,[],[],"[B00004U9V2, B00GHX7H0A, B00FRERO7G, B00R68QXC...",{' Product Dimensions: ': '2.6 x 2.6 ...,Luxury Beauty,,,$15.99,B00005A77F,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
191,[],,[This liquid soap with convenient pump dispens...,,"Crabtree &amp; Evelyn Hand Soap, Gardeners, 10...",[],,,[],[],"[B00004U9V2, B00GHX7H0A, B00FRERO7G, B00R68QXC...",{' Product Dimensions: ': '2.6 x 2.6 ...,Luxury Beauty,,,$15.99,B00005A77F,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [16]:
di_meta_title = df_meta[['asin','title']].to_dict('split')

In [17]:
li_revs = df_reviews.to_dict('split')

In [18]:
li_revs['data'][:5]

[['B00004U9V2',
  "I bought two of these 8.5 fl oz hand cream, and neither of the dispensers work.  The hand cream is wonderful, but very thick, so I don't see I can get much out by shaking it out, since the dispensers seem to be non-operable."],
 ['B00004U9V2',
  'Believe me, over the years I have tried many, many different hand creams. I am one of those people whose hands get so dry they have little cracks all over them.\n\nOf all the hand creams, this is the best. It really moisturizes but doesn\'t leave your hands feeling greasy. And it lasts. I think a lot of lotions just have too much water in them. It has a very nice, subtle scent. I have to avoid a lot of lotions just because the scent is too strong!\n\nI am overall very "frugal" but I think this cream is worth the money.'],
 ['B00004U9V2', 'Great hand lotion'],
 ['B00004U9V2', 'This is the best for the severely dry skin on my hands'],
 ['B00004U9V2', 'The best non- oily hand cream ever. It heals overnight.']]

In [19]:
di_meta_title['data'][:5]

[['B00004U9V2',
  "Crabtree &amp; Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ"],
 ['B0000531EN', 'AHAVA Bath Salts'],
 ['B0000532JH', 'AHAVA Dead Sea Mineral Mud, 8.5 oz, Pack of 4'],
 ['B00005A77F', 'Crabtree &amp; Evelyn Hand Soap, Gardeners, 10.1 fl. oz.'],
 ['B00005NDTD', 'Soy Milk Hand Crme']]

In [46]:
def connect_asins(li_revs: dict, meta = False) -> dict:
    dict_revs = {}
    revs_app = []
    for elem in tqdm(li_revs['data']):
        if elem[1] != 'nan':
            if elem[0] not in list(dict_revs.keys()):
                revs_app = [elem[1]]
            else:
                revs_app.append(elem[1])
            if len(revs_app) < 50:
                dict_revs[elem[0]] = revs_app
            else:
                dict_revs[elem[0]] = revs_app[:50]
            if meta == True:
                revs_app = []
    return dict_revs

In [47]:
dict_revs = connect_asins(li_revs)


  0%|          | 0/574628 [00:00<?, ?it/s]

In [22]:
import re
from collections import Counter

In [ ]:
!pip install --quiet spacy
!spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

Мне важно было смотреть на все отзывы для одного товара. Ощущение, что так наши частотные слова будут чище, ведь мы изначально смотрим не на весь корпус, а только на один товар. Плюс так мы сможем использовать биграммы без применения tf-idf.

In [34]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [38]:
def preprocess_text(text: str) -> str:
    lemmas = []
    for word in text.split():
        lemmas.append(lemmatizer.lemmatize(word))
    text = [word.lower().strip(punctuation) for word in lemmas]
    spaces = ['', ' ', '\n', '\t']
    text = [word for word in text if word not in spaces]
    stop_words = set(stopwords)
    filtered_sentence = [w for w in text if not w.lower() in stop_words]
    text = ' '.join(filtered_sentence)
    text = re.sub(r"\s+", " ", text)
    return text

In [48]:
dict_revs_preproc = {}
for key, value in tqdm(dict_revs.items()):
    valuee = []
    for text in value:
        if type(text)!=float:
            text = preprocess_text(text)
            text = nlp(text)
            for tok in text:
                valuee.append(tok.lemma_)
    all_vals = Counter(valuee)
    bigrams = zip(valuee, valuee[1:])
    counts = Counter(bigrams)
    all_counts = all_vals + counts

    all_counts = all_counts.most_common(10)
    dict_revs_preproc[key] = all_counts

  0%|          | 0/12120 [00:00<?, ?it/s]

In [26]:
li_meta = df_meta['title'].tolist()

In [91]:
li_revs = df['reviewText'].tolist()

In [27]:
li_meta[1]

'AHAVA Bath Salts'

In [84]:
for elem in tqdm(li_meta):
    text = preprocess_text(elem)
    text = nlp(text)
    for tok in text:
        if tok.pos_ == "NOUN":
            valuee.append(tok.lemma_)
    bigrams = zip(valuee, valuee[1:])
    counts = Counter(bigrams)
    all_vals = Counter(valuee) 
    all_counts = all_vals + counts

  0%|          | 0/12299 [00:00<?, ?it/s]

In [182]:
li_common_titles = []
for i,j in all_counts.most_common(50):
    li_common_titles.append(i)

In [86]:
li_all = list(dict_revs_preproc.values())

In [88]:
set_words = []
for elem in li_all:
    for i,j in elem:
        if i!= '-':
            if type(i) == tuple:
                i = ' '.join(i)
            set_words.append(i)
set_words = set(set_words)

Пересечение самых частотных тайтлов и слов в отзывах должно давать названия

In [183]:
set_names = set(li_common_titles)& set_words

In [184]:
set_names

{'1',
 '1.7',
 '2',
 'beauty',
 'body',
 'brush',
 'butter',
 'care',
 'cleanser',
 'cleansing',
 'collection',
 'color',
 'conditioner',
 'crabtree',
 'cream',
 'de',
 'eau',
 'eye',
 'face',
 'fl',
 'gel',
 'hair',
 'hand',
 'kit',
 "l'occitane",
 'lip',
 'lotion',
 'man',
 'mask',
 'mineral',
 'moisturizer',
 'nail',
 'oil',
 'ounce',
 'oz',
 'parfum',
 'powder',
 'serum',
 'set',
 'shampoo',
 'skin',
 'spf',
 'spray',
 'treatment',
 'wash'}

Получилось немного мусора, поэтому попробуем достать частотные сочетания через tf-idf

In [171]:
import gensim

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [188]:
valuee_st = []

for elem in tqdm(li_revs):
    valuee = []
    if type(elem) != float:
        text = preprocess_text(elem)
        text = nlp(text)
        for tok in text:
            if tok.pos_ == "NOUN":
                valuee.append(tok.lemma_)
    valuee_st.append(' '.join(valuee))

  0%|          | 0/574628 [00:00<?, ?it/s]

In [189]:
corpus = [text.split(' ') for text in valuee_st]

model = gensim.models.Word2Vec(corpus, window=2)

In [190]:
synonyms = []

for elem in set_names:
  if model.wv.__contains__(elem):
    for syn in model.wv.most_similar(elem, topn=3):
      synonyms.append(syn[0])

In [191]:
synonyms

['organic',
 'skincare',
 'href="/clinique',
 'eyelid',
 'undereye',
 'puffiness',
 'session',
 'remedy',
 'treat',
 'strand',
 'wavy',
 'bouncy',
 'muppet',
 'lover',
 'edition',
 'loccitane',
 'crabtree',
 'sabon',
 'veil',
 'powdery',
 'blend',
 'colour',
 'shade',
 'hue',
 'retinol',
 'c',
 'booster',
 'mud',
 'masque',
 'fango',
 'washing',
 'rinse',
 'cleanse',
 'sunblock',
 'sunscreen',
 'spectrum',
 'chapstick',
 'lipstick',
 'gloss',
 'toilette',
 'issey',
 'roma',
 'cleanse',
 'cleanser',
 'foaming',
 'tonic',
 'prep',
 'wet',
 'cleanse',
 'cleansing',
 'cleaner',
 'pouch',
 'compartment',
 'compact',
 'lacoste',
 'eternity',
 'euphoria',
 'emollient',
 'aveeno',
 'moisturizer',
 'turbo',
 'loofah',
 'remainder',
 'cocoa',
 'palmer',
 'emollient',
 'creme',
 'aveeno',
 'emollient',
 'guy',
 'dude',
 'gentleman',
 'rosacea',
 'redness',
 'face',
 'bi',
 'ex',
 'pre',
 'skin',
 'cloth',
 'decollete',
 'fingernail',
 'chip',
 'manicure',
 'palm',
 'fingertip',
 'rub',
 'bristle'

Составим полный списко названий

In [198]:
set_names

{'beauty',
 'body',
 'brush',
 'butter',
 'care',
 'cleanser',
 'cleansing',
 'collection',
 'color',
 'conditioner',
 'crabtree',
 'cream',
 'de',
 'eau',
 'eye',
 'face',
 'fl',
 'gel',
 'hair',
 'hand',
 'kit',
 "l'occitane",
 'lip',
 'lotion',
 'man',
 'mask',
 'mineral',
 'moisturizer',
 'nail',
 'oil',
 'ounce',
 'oz',
 'parfum',
 'powder',
 'serum',
 'set',
 'shampoo',
 'skin',
 'spf',
 'spray',
 'treatment',
 'wash'}

In [197]:
li_names = ['brush', 'butter', 'cleanser', 'conditioner', 'cream', 'gel', 'hair','lip', 'lotion',  'mask', 'moisturizer', 'nail', 'oil', 'parfum', 'powder','serum','shampoo',
            'spray', 'booster','sunscreen',  'sebum', 'mist','tonic','toilette', 'chapstick','lipstick']

Финальный список названий.

In [199]:
li_names

['brush',
 'butter',
 'cleanser',
 'conditioner',
 'cream',
 'gel',
 'hair',
 'lip',
 'lotion',
 'mask',
 'moisturizer',
 'nail',
 'oil',
 'parfum',
 'powder',
 'serum',
 'shampoo',
 'spray',
 'booster',
 'sunscreen',
 'sebum',
 'mist',
 'tonic',
 'toilette',
 'chapstick',
 'lipstick']

## Собираем n-граммы

In [200]:
from nltk.util import ngrams
from nltk.collocations import *

In [203]:
bigrams = []
for text in corpus:
    bigr = ngrams(text, 2)
    for b in bigr:
        bigrams.append(b)
    

In [205]:
len(bigrams)

4117562

In [206]:
bigrams_count = Counter(bigrams).most_common()

In [ ]:
bigrams_count

In [208]:
bigrams_keywords = []

for elem in bigrams:
  for word in set_names:
    if word in elem[0]:
      bigrams_keywords.append(elem)

In [221]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([text.split(' ') for text in valuee_st])
finder.apply_word_filter(lambda x: x not in set_names)

In [220]:
bigram_measures

## Ранжирование

In [222]:
finder.nbest(bigram_measures.likelihood_ratio, 15)

[('shampoo', 'conditioner'),
 ('face', 'wash'),
 ('body', 'wash'),
 ('skin', 'care'),
 ('hand', 'cream'),
 ('hair', 'skin'),
 ('skin', 'hair'),
 ('eye', 'cream'),
 ('body', 'lotion'),
 ('conditioner', 'hair'),
 ('nail', 'nail'),
 ('wash', 'face'),
 ('hair', 'hair'),
 ('lip', 'lip'),
 ('face', 'hair')]

In [223]:
finder.nbest(bigram_measures.student_t, 15)

[('shampoo', 'conditioner'),
 ('face', 'wash'),
 ('skin', 'care'),
 ('hand', 'cream'),
 ('hair', 'hair'),
 ('eye', 'cream'),
 ('body', 'wash'),
 ('conditioner', 'hair'),
 ('shampoo', 'hair'),
 ('hair', 'spray'),
 ('color', 'color'),
 ('body', 'lotion'),
 ('nail', 'nail'),
 ('wash', 'face'),
 ('wash', 'hair')]

In [224]:
finder.nbest(bigram_measures.pmi, 15)

[('eau', 'de'),
 ('fl', 'oz'),
 ('man', 'eau'),
 ('de', 'fl'),
 ('eau', 'ounce'),
 ('ounce', 'ounce'),
 ('fl', 'fl'),
 ('mineral', 'powder'),
 ('oz', 'oz'),
 ('body', 'wash'),
 ('shampoo', 'conditioner'),
 ('body', 'butter'),
 ('spf', 'spf'),
 ('parfum', 'parfum'),
 ('cleansing', 'conditioner')]

Первые две отработали примерно одинаково, но мы видим, что третья сильно от них отличается. Хотя, можно увидеть body butter, что нам подходит. 

## Выведем топ